In [4]:
import sys
print(sys.executable)

/opt/homebrew/opt/python@3.11/bin/python3.11


In [1]:
!pip3 install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip


In [3]:

import os
import xml.etree.ElementTree as ET
import pandas as pd

ANDROID_NS = "http://schemas.android.com/apk/res/android"
MANIFEST_DIR = "android-manifest-files"





In [4]:
def parse_manifest(filepath):
    try:
        tree = ET.parse(filepath)
        root = tree.getroot()
        app_package = root.attrib.get("package")
        queried_packages = []

        for queries in root.findall("queries"):
            for pkg in queries.findall("package"):
                name = pkg.attrib.get(f"{{{ANDROID_NS}}}name")
                if name:
                    queried_packages.append(name)

        return app_package, queried_packages
    except:
        return None, []


In [5]:
results = []

for root, _, files in os.walk(MANIFEST_DIR):
    for file in files:
        if file.endswith("AndroidManifest.xml"):
            path = os.path.join(root, file)
            app, queries = parse_manifest(path)
            if app and queries:
                for q in queries:
                    results.append({
                        "app_package": app,
                        "queried_package": q
                    })

print(f"Extracted {len(results)} app → queried app relationships.")




Extracted 3424 app → queried app relationships.


In [7]:
df = pd.DataFrame(results)
df.head(1000)



,app_package,queried_package
0,in.startv.hotstar,net.one97.paytm
1,in.startv.hotstar,com.google.android.gms.policy_cast_dynamite
2,in.startv.hotstar,androidx.test.orchestrator
3,in.startv.hotstar,androidx.test.services
4,in.startv.hotstar,com.google.android.apps.common.testing.services
...,...,...
995,com.whizdm.moneyview.loans,com.loangopro.bestloanapp
996,com.whizdm.moneyview.loans,com.Loanindiaon
997,com.whizdm.moneyview.loans,com.loanklub.cwtapp
998,com.whizdm.moneyview.loans,com.loanpelite.loanapp


In [8]:
df.to_csv("app_query_relationships.csv", index=False)
df.to_json("app_query_relationships.json", orient="records", indent=2)



In [9]:
import pandas as pd

df = pd.read_csv("app_query_relationships.csv")

In [11]:
df[df["app_package"].str.contains("axis", case=False)]

,app_package,queried_package
604,com.axis.mobile,com.tencent.mm
605,com.axis.mobile,com.whatsapp
606,com.axis.mobile,us.zoom.videomeetings
607,com.axis.mobile,com.google.android.gms.policy_cast_dynamite
608,com.axis.mobile,com.google.android.apps.maps
609,com.axis.mobile,com.facebook.katana
610,com.axis.mobile,com.instagram.android
611,com.axis.mobile,com.facebook.lite
612,com.axis.mobile,com.samsung.android.mapsagent
613,com.axis.mobile,com.android.vending


In [12]:
queried = df["queried_package"].dropna().unique()
pd.Series(queried).to_csv("all_queried_packages.csv", index=False, header=["package"])


In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import ace_tools as tools

# Paths
rel_path = Path("app_query_relationships.csv")
cat_path = Path("app_categories.csv")

# Load data
rel_df = pd.read_csv(rel_path)
cat_df = pd.read_csv(cat_path)

# Map package -> category and subcategory
cat_map = cat_df.set_index("package")[["category", "subcategory"]]
rel_df = rel_df.merge(cat_map, left_on="app_package", right_index=True, how="left", suffixes=("", "_source"))
rel_df = rel_df.merge(cat_map, left_on="queried_package", right_index=True, how="left", suffixes=("", "_target"))

# Rename for clarity
rel_df.rename(columns={"category": "source_category",
                       "subcategory": "source_subcategory",
                       "category_target": "target_category",
                       "subcategory_target": "target_subcategory"}, inplace=True)

# Fill missing with 'unknown'
rel_df[["source_category", "source_subcategory", "target_category", "target_subcategory"]] = rel_df[
    ["source_category", "source_subcategory", "target_category", "target_subcategory"]
].fillna("unknown")

# ============== INSIGHT 1: Category → Category matrix (% of queries) ==============
cat_matrix = rel_df.groupby(["source_category", "target_category"]).size().unstack(fill_value=0)
cat_pct = cat_matrix.div(cat_matrix.sum(axis=1), axis=0) * 100
cat_pct_rounded = cat_pct.round(1)

# Display matrix to user
tools.display_dataframe_to_user("Category_to_Category_%", cat_pct_rounded)

# ============== INSIGHT 2: Top querying apps ==============
top_query_apps = rel_df.groupby("app_package").size().sort_values(ascending=False).head(20)
top_query_apps_df = top_query_apps.reset_index().rename(columns={0: "num_queries"})
tools.display_dataframe_to_user("Top_20_Querying_Apps", top_query_apps_df)

# ============== INSIGHT 3: Bar chart of target categories overall ==============
target_counts = rel_df["target_category"].value_counts().head(10)
plt.figure(figsize=(8,6))
target_counts.plot(kind='bar')
plt.title("Top Queried Categories (overall)")
plt.ylabel("Number of queries")
plt.xlabel("Target Category")
plt.tight_layout()
plt.show()




ModuleNotFoundError: No module named 'ace_tools'